In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
import sys
from time import time
# from mpl_toolkits.basemap import Basemap

In [2]:
base_dir = "../dataset"
# nc.Dataset(f"{base_dir}/")
df_locations = pd.read_csv(f"{base_dir}/SKNlocations.csv")
df_data = pd.read_excel(f"{base_dir}/FilledDataset2012.xlsx", sheet_name="Data_in")

In [3]:
# Delete this cell
df_test = pd.read_csv(f"{base_dir}/dataset_v2.csv")

In [4]:
df_test.shape

(90960, 24)

In [5]:
df_data.head(3)

,SKN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1.0,1920,4.76,0.23,0.005188,0.43,0.30,NaN,NaN,0,0,0.87,NaN,NaN
1,1.0,1921,7.78,1.29,2.946572,0.58,0.27,NaN,NaN,0.58,0,0.44,NaN,NaN
2,1.0,1922,4.02,2.75,6.053961,1.63,0.03,NaN,NaN,0.04,0.74,0.95,NaN,NaN


In [6]:
df_locations.head(3)

,SKN,Name,Lat_DD,Lon_DD
0,1.0,KALAE,18.916176,-155.674994
1,1.1,MORSE FIELD,18.913680,-155.680550
2,1.2,KALAE S TRK STA,18.938669,-155.680549


## Conevrt df_data into cell-based matrix

In [7]:
X = []
for index, row in df_data.iterrows():
    if row.Year < 1948:
        # No need to keep data older than 1948 becase no data exists in netCDF files
        continue
    for i, cell in enumerate(row[2:]):
        X.append([row.SKN, row.Year, i + 1, cell])

In [8]:
df_data_by_cell = pd.DataFrame(X, columns = ["skn", "year", "month", "data_in"]).dropna()
df_data_by_cell = df_data_by_cell.replace(r'^\s*$', np.nan, regex=True).dropna()

In [9]:
df_data_by_cell.head(3)

,skn,year,month,data_in
0,1.0,1948,1,3.2
1,1.0,1948,2,1.33
2,1.0,1948,3,2.31


In [10]:
# Merge
df_data_w_coord = df_data_by_cell.merge(right=df_locations, left_on="skn", right_on="SKN")
df_data_w_coord["time"] = df_data_w_coord.apply(lambda x: f"{x.year}-{x.month}-01", axis=1)
df_data_w_coord.head(3)

,skn,year,month,data_in,SKN,Name,Lat_DD,Lon_DD,time
0,1.0,1948,1,3.2,1.0,KALAE,18.916176,-155.674994,1948-1-01
1,1.0,1948,2,1.33,1.0,KALAE,18.916176,-155.674994,1948-2-01
2,1.0,1948,3,2.31,1.0,KALAE,18.916176,-155.674994,1948-3-01


## Load the cdf files

In [11]:
# ds = xr.open_dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc")
lat_hawaii = [15, 17.5, 20, 22.5, 25]
lon_hawaii = np.array([-162.5, -160, -157.5, -155, -152.5]) + 360

ds_air2m = xr.open_dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_air1000_500 = xr.open_dataset(f"{base_dir}/air.1000-500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_hgt500 = xr.open_dataset(f"{base_dir}/hgt500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_hgt1000 = xr.open_dataset(f"{base_dir}/hgt1000.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_omega500 = xr.open_dataset(f"{base_dir}/omega500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_pottemp_1000_500 = xr.open_dataset(f"{base_dir}/pottmp.1000-500.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_pottemp_1000_850 = xr.open_dataset(f"{base_dir}/pottmp.1000-850.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_pwtr = xr.open_dataset(f"{base_dir}/pwtr.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_u700 = xr.open_dataset(f"{base_dir}/shum_x_uwnd.700.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_u925 = xr.open_dataset(f"{base_dir}/shum_x_uwnd.925.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_v700 = xr.open_dataset(f"{base_dir}/shum_x_vwnd.700.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_v950 = xr.open_dataset(f"{base_dir}/shum_x_vwnd.925.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_shum700 = xr.open_dataset(f"{base_dir}/shum700.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_shum925 = xr.open_dataset(f"{base_dir}/shum925.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_skt = xr.open_dataset(f"{base_dir}/skt.mon.mean.regridded.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]
ds_slp = xr.open_dataset(f"{base_dir}/slp.mon.mean.nc").loc[dict(lat=lat_hawaii, lon=lon_hawaii)]

datasets = [ # list of tuples. (dataset object, label, attribute string in ds)
    (ds_air2m, "air2m", "air"),
    (ds_air1000_500, "air1000_500", "air"),
    (ds_hgt500, "hgt500", "hgt"),
    (ds_hgt1000, "hgt1000", "hgt"),
    (ds_omega500, "omega500", "omega"),
    (ds_pottemp_1000_500, "pottemp1000-500", "pottmp"),
    (ds_pottemp_1000_850, "pottemp1000-850", "pottmp"),
    (ds_pwtr, "pr_wtr", "pr_wtr"),
    (ds_u700, "shum-uwnd-700", "shum"),
    (ds_u925, "shum-uwnd-925", "shum"),
    (ds_v700, "shum-vwnd-700", "shum"),
    (ds_v950, "shum-vwnd-950", "shum"),
    (ds_shum700, "shum700", "shum"),
    (ds_shum925, "shum925", "shum"),
    (ds_skt, "skt", "skt"),
    (ds_slp, "slp", "slp")
]
# combine all the cdf data

In [ ]:
new_data = []
for year in df_data["Year"].unique():
    if year < 1948: continue
    for month in [i + 1 for i in range(12)]:
        row = {}
        row["time"] = f"{year}-{month}-01"
        for ds, label, attribute in datasets:
            row[label] = ds.loc[dict(time=f"{year}-{month}-01")][attribute]
        new_data.append(row)

In [ ]:
df_cdf_layers_by_month = pd.DataFrame(new_data)
df_cdf_layers_by_month.head(3)

## Merge

In [ ]:
df_data_w_coord.head(3)

In [ ]:
df_complete = df_data_w_coord.merge(right=df, left_on="time", right_on="time")

In [ ]:
df_complete.head(3)

In [ ]:
df_complete_clean = df_complete.drop(labels=["SKN", "time"], axis=1).rename(columns={"Lat_DD": "lat", "Lon_DD": "lon"})

In [ ]:
df_complete_clean.head(3)

In [ ]:
df_complete_clean.to_csv(f"{base_dir}/dataset_v2.csv")